# Competición Kaggle 2


### 1: Importar librerias

In [23]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV


### 2: Cargar los datos

In [24]:
# Cargar archivos 
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

# Verificar que se cargaron correctamente
print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)


Train shape: (105000, 12)
Test shape: (45000, 11)


In [25]:
train_df.head()

,ID,RevolvingUtilizationOfUnsecuredLines,Age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,SeriousDlqin2yrs
0,9580,0.668999,58,2,0.449504,3425.0,9,1,1,1,1.0,0
1,39755,0.015922,71,0,6.000000,NaN,5,0,0,0,0.0,0
2,118799,0.183062,52,1,0.035593,5000.0,9,0,0,0,0.0,0
3,16489,0.162301,77,0,0.227886,2000.0,8,0,0,0,0.0,0
4,149857,0.404199,30,0,0.026010,5843.0,4,0,0,0,0.0,0


In [26]:
test_df.head()

,ID,RevolvingUtilizationOfUnsecuredLines,Age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,129460,1.000000,21,0,8.000000,NaN,0,0,0,0,NaN
1,134018,0.009878,38,0,0.229978,10500.0,10,0,1,0,1.0
2,86523,0.276836,70,0,1914.000000,NaN,23,0,1,0,0.0
3,138466,0.045413,75,0,452.000000,NaN,4,0,0,0,0.0
4,143905,0.000000,82,0,0.000000,NaN,5,0,0,0,0.0


In [27]:
train_df.describe()

,ID,RevolvingUtilizationOfUnsecuredLines,Age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,SeriousDlqin2yrs
count,105000.000000,105000.000000,105000.000000,105000.000000,105000.000000,8.416400e+04,105000.000000,105000.000000,105000.000000,105000.000000,102236.000000,105000.000000
mean,75006.458152,5.378324,52.325610,0.409352,352.044192,6.703641e+03,8.459952,0.254619,1.019010,0.228762,0.757933,0.066514
std,43315.742022,201.573457,14.766425,4.056717,1820.229318,1.622288e+04,5.134329,4.032506,1.131065,4.017864,1.115273,0.249180
min,1.000000,0.000000,21.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,37590.750000,0.029974,41.000000,0.000000,0.174950,3.400000e+03,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,74941.500000,0.154252,52.000000,0.000000,0.366061,5.400000e+03,8.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,112542.500000,0.556035,63.000000,0.000000,0.870083,8.250000e+03,11.000000,0.000000,2.000000,0.000000,1.000000,0.000000
max,149999.000000,29110.000000,109.000000,98.000000,329664.000000,3.008750e+06,58.000000,98.000000,54.000000,98.000000,20.000000,1.000000


In [28]:
test_df.describe()

,ID,RevolvingUtilizationOfUnsecuredLines,Age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
count,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,36105.000000,45000.000000,45000.000000,45000.000000,45000.000000,43840.000000
mean,74986.597644,7.612037,52.224267,0.448289,355.247139,6592.315718,8.435978,0.292467,1.016444,0.267511,0.755566
std,43268.443146,336.328241,14.784475,4.494203,2472.160315,8704.001581,5.172986,4.472190,1.126756,4.459097,1.114661
min,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,37318.750000,0.029653,41.000000,0.000000,0.175327,3400.000000,5.000000,0.000000,0.000000,0.000000,0.000000
50%,75119.500000,0.154037,52.000000,0.000000,0.367499,5400.000000,8.000000,0.000000,1.000000,0.000000,0.000000
75%,112379.250000,0.565636,63.000000,0.000000,0.863632,8216.000000,11.000000,0.000000,2.000000,0.000000,1.000000
max,150000.000000,50708.000000,109.000000,98.000000,326442.000000,702500.000000,56.000000,98.000000,32.000000,98.000000,10.000000


### 3: Separar variables

In [29]:
# Seleccionamos la columna objetivo (target) del conjunto de entrenamiento
# Esta columna indica si una persona tuvo un atraso en los pagos (1) o no (0)
y_train = train_df['SeriousDlqin2yrs']

# Eliminamos la columna objetivo del conjunto de entrenamiento para quedarnos solo con las variables predictoras (features)
# Esto es lo que se usará como entrada del modelo
# 'SeriousDlqin2yrs' es la variable objetivo y no debe incluirse en el conjunto de entrenamiento
X_train = train_df.drop('SeriousDlqin2yrs', axis=1)

# Hacemos una copia del conjunto de prueba
X_test = test_df.copy()

# Guardamos la columna 'ID' del conjunto de prueba
# Esto es útil para identificar a qué registro pertenece cada predicción en la etapa final
test_ids = X_test['ID']



### 4: Limpieza - Reemplazo de valores atípicos y NaNs

In [30]:
# Lista de columnas que contienen valores atípicos (96 y 98)
# Estas columnas indican la cantidad de veces que un cliente se atrasó en sus pagos
# 96 y 98 no son valores posibles reales
outlier_cols = [
    'NumberOfTime30-59DaysPastDueNotWorse',   # Número de veces con atraso entre 30 y 59 días
    'NumberOfTimes90DaysLate',                # Número de veces con atraso de 90 días
    'NumberOfTime60-89DaysPastDueNotWorse'    # Número de veces con atraso entre 60 y 89 días
]

# Reemplazamos los valores 96 y 98 por NaN (valores nulos) en cada columna afectada
# Se hace tanto en los datos de entrenamiento como en los datos de prueba
for col in outlier_cols:
    X_train[col] = X_train[col].replace({96: np.nan, 98: np.nan})
    X_test[col] = X_test[col].replace({96: np.nan, 98: np.nan})



### 5: Imputación de valores faltantes con la mediana

In [31]:
# Calcular la mediana de cada columna en el conjunto de entrenamiento
# La mediana es una medida robusta que no se ve afectada por valores extremos
medians = X_train.median()

# Rellenar los valores faltantes (NaN) en el conjunto de entrenamiento usando las medianas calculadas
# inplace=True indica que los cambios se aplican directamente al DataFrame (no se crea una copia)
X_train.fillna(medians, inplace=True)

# Rellenar los valores faltantes en el conjunto de prueba usando las mismas medianas del entrenamiento
# Importante: usamos las medianas de entrenamiento para evitar filtración de datos del test
X_test.fillna(medians, inplace=True)

# Verificar cuántos valores faltantes quedan en X_train y X_test
# isnull().sum().sum() cuenta cuántos valores NaN hay en total en cada conjunto
print("Faltantes en X_train:", X_train.isnull().sum().sum())
print("Faltantes en X_test:", X_test.isnull().sum().sum())



Faltantes en X_train: 0
Faltantes en X_test: 0


### 6: Eliminar columna 'ID'

In [32]:
# Eliminar la columna 'ID' del conjunto de entrenamiento
# axis=1 indica que se elimina una columna, inplace=True aplica el cambio directamente
X_train.drop('ID', axis=1, inplace=True)

# Eliminar la columna 'ID' del conjunto de prueba
X_test.drop('ID', axis=1, inplace=True)


### 7: Calcular scale_pos_weight

In [33]:
# Contar cuántas instancias hay de cada clase en la variable objetivo
neg, pos = y_train.value_counts()

# Calcular la proporción entre clases
# Ajustar el modelo en caso de que haya mucho más de una clase que de la otra
scale_pos_weight_value = neg / pos

# Mostrar el valor calculado de scale_pos_weight (número con dos decimales)
# Este valor puede ser usado al entrenar modelos como XGBoost para compensar el desbalance de clases
print(f"scale_pos_weight: {scale_pos_weight_value:.2f}")



scale_pos_weight: 14.03


- hay aproximadamente 14 veces más ejemplos de la clase negativa (0) que de la clase positiva (1)

### 8: Definir modelo y espacio de búsqueda

In [34]:
# Crear el modelo base usando LightGBM para clasificación binaria
# Este modelo aún no se entrena, solo se define con sus parámetros iniciales
model = lgb.LGBMClassifier(
    objective='binary',             # Tipo de problema: clasificación binaria (0 o 1)
    metric='auc',                   # Métrica a optimizar: AUC (área bajo la curva ROC)
    random_state=42,                # Semilla para reproducibilidad
    n_jobs=-1,                      # Usa todos los núcleos disponibles para entrenamiento
    scale_pos_weight=scale_pos_weight_value  # Peso para la clase positiva, útil si el dataset está desbalanceado
)

# Definir el espacio de búsqueda de hiperparámetros
# Esto se usará para encontrar la mejor combinación de parámetros mediante búsqueda automatizada
param_grid = {
    'n_estimators': [500, 1000, 1500],       # Número de árboles (más = más precisión, pero más lento)
    'learning_rate': [0.01, 0.05, 0.1],      # Tasa de aprendizaje (más bajo = más lento pero más preciso)
    'num_leaves': [20, 31, 50],              # Número máximo de hojas por árbol (controla la complejidad)
    'max_depth': [-1, 10, 20],               # Profundidad máxima del árbol (-1 = sin límite)
    'min_child_samples': [20, 50, 100],      # Mínimo de muestras en un nodo hijo para hacer una división
    'subsample': [0.7, 0.8, 0.9, 1.0],       # Fracción de datos a usar por árbol (para evitar overfitting)
    'colsample_bytree': [0.7, 0.8, 0.9, 1.0],# Fracción de columnas a usar por árbol (tipo de regularización)
    'reg_alpha': [0, 0.1, 0.5],              # Regularización L1 (reduce complejidad del modelo)
    'reg_lambda': [0, 0.1, 0.5]              # Regularización L2 (también ayuda a prevenir overfitting)
}



### 9: Búsqueda de hiperparámetros con CV

In [35]:
# Configurar RandomizedSearchCV para encontrar la mejor combinación de hiperparámetros
random_search = RandomizedSearchCV(
    estimator=model,                     # Modelo base (en este caso, un LGBMClassifier)
    param_distributions=param_grid,      # Espacio de búsqueda de hiperparámetros definido antes
    n_iter=10,                           # Número de combinaciones aleatorias que se probarán (10 modelos)
    scoring='roc_auc',                   # Métrica a optimizar: AUC (ideal para clasificación desbalanceada)
    cv=StratifiedKFold(n_splits=3),      # Validación cruzada estratificada en 3 partes (mantiene proporción de clases)
    verbose=1,                           # Muestra progreso en pantalla
    random_state=42,                     # Semilla para reproducibilidad
    n_jobs=-1                            # Usa todos los núcleos disponibles para entrenar más rápido
)

# Entrenar el modelo con búsqueda de hiperparámetros
# Esto probará 10 combinaciones distintas y seleccionará la mejor
random_search.fit(X_train, y_train)

# Mostrar los mejores parámetros encontrados
print("Mejores parámetros:", random_search.best_params_)

# Mostrar el mejor puntuaje promedio (AUC) obtenido en la validación cruzada
print(f"Mejor AUC promedio: {random_search.best_score_:.4f}")


Fitting 3 folds for each of 10 candidates, totalling 30 fits
[LightGBM] [Info] Number of positive: 6984, number of negative: 98016
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0,003139 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 965
[LightGBM] [Info] Number of data points in the train set: 105000, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0,066514 -> initscore=-2,641509
[LightGBM] [Info] Start training from score -2,641509
Mejores parámetros: {'subsample': 0.8, 'reg_lambda': 0.1, 'reg_alpha': 0.5, 'num_leaves': 31, 'n_estimators': 500, 'min_child_samples': 100, 'max_depth': 10, 'learning_rate': 0.01, 'colsample_bytree': 0.7}
Mejor AUC promedio: 0.8622


### 10: Entrenamiento final

In [36]:
# Crear el modelo final utilizando los mejores parámetros encontrados en la búsqueda aleatoria
# **random_search.best_params_** carga los parámetros óptimos para el modelo
final_model = lgb.LGBMClassifier(
    objective='binary',                 # Clasificación binaria (0 o 1)
    metric='auc',                       # Métrica a optimizar: AUC (ideal para desbalanceo de clases)
    random_state=42,                    # Semilla para reproducibilidad
    n_jobs=-1,                          # Utilizar todos los núcleos disponibles para acelerar el entrenamiento
    scale_pos_weight=scale_pos_weight_value,  # Ajuste para clases desbalanceadas
    **random_search.best_params_        # Los mejores parámetros encontrados por RandomizedSearchCV se aplican aquí
)

# Entrenar el modelo con todos los datos del conjunto de entrenamiento
final_model.fit(X_train, y_train)


[LightGBM] [Info] Number of positive: 6984, number of negative: 98016
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0,003465 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 965
[LightGBM] [Info] Number of data points in the train set: 105000, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0,066514 -> initscore=-2,641509
[LightGBM] [Info] Start training from score -2,641509


LGBMClassifier(colsample_bytree=0.7, learning_rate=0.01, max_depth=10,
               metric='auc', min_child_samples=100, n_estimators=500, n_jobs=-1,
               objective='binary', random_state=42, reg_alpha=0.5,
               reg_lambda=0.1, scale_pos_weight=14.034364261168385,
               subsample=0.8)

### 11: Predicciones

In [37]:
# Predicción de probabilidades sobre el conjunto de test
# `predict_proba` devuelve un array con las probabilidades para cada clase (0 y 1)
# `[:, 1]` selecciona la columna de probabilidades para la clase positiva (1)
y_pred_proba = final_model.predict_proba(X_test)[:, 1]

# Mostrar las primeras 10 predicciones de probabilidad
# Esto muestra las probabilidades de que cada muestra en X_test pertenezca a la clase positiva (1)
print("Primeras 10 predicciones de probabilidad:", y_pred_proba[:10])



Primeras 10 predicciones de probabilidad: [0.76625723 0.09382167 0.09692313 0.06760889 0.04413982 0.92927743
 0.04163162 0.60982751 0.21954878 0.29182939]


###  12: Archivo de envío

In [38]:
submission = pd.DataFrame({
    'ID': test_ids,
    'SeriousDlqin2yrs': y_pred_proba
})

submission.to_csv('sample_submission.csv', index=False)
print("Archivo de envío 'sample_submission.csv' creado.")


Archivo de envío 'sample_submission.csv' creado.
